# Tags analysis

In [11]:
import json

with open("../../data/tags/Sorceleur - L'Integrale - Andrzej Sapkowski.json") as f:
    data = json.load(f)

In order to validate an entity_group : if the best represented class represents only a small percentage, double check with a LLM

In [12]:
classes = sorted(list(data['tags'].keys()))
print(classes)

entities = dict()
for eg_key, eg_value in data['tags'].items():
    for tag_key, tag_value in eg_value.items():
        if tag_key not in entities.keys():
            entities[tag_key] = {}
        entities[tag_key][eg_key] = {
            'count': tag_value['count'],
            'median': tag_value['median'],
        }

entity_name_entity_groups_association = []
for entity_name, entity_info in entities.items():
    total_tags = sum([info['count'] for info in entity_info.values()])
    eg_tuples = []
    for entity_group, entity_group_info in entity_info.items():
        eg_tuples.append((entity_group, entity_group_info['count'], entity_group_info['count']/total_tags))
    eg_tuples = sorted(eg_tuples, reverse=True, key=lambda x: x[2])
    entity_name_entity_groups_association.append((entity_name, total_tags, eg_tuples))

entity_name_entity_groups_association = sorted(entity_name_entity_groups_association, reverse=True, key=lambda x: x[1])

for elem in entity_name_entity_groups_association:
    print(f'{elem[0]} : {" ".join([str(t) for t in elem[2]])}')


['CONSUMER_GOOD', 'EVENT', 'LOCATION', 'ORGANIZATION', 'OTHER', 'PERSON', 'WORK_OF_ART']
Geralt : ('PERSON', 3652, 0.9991792065663475) ('WORK_OF_ART', 2, 0.0005471956224350205) ('LOCATION', 1, 0.00027359781121751026)
Ciri : ('PERSON', 2193, 0.9986338797814208) ('LOCATION', 1, 0.0004553734061930783) ('OTHER', 1, 0.0004553734061930783) ('WORK_OF_ART', 1, 0.0004553734061930783)
Jaskier : ('PERSON', 1465, 0.9938941655359566) ('CONSUMER_GOOD', 6, 0.004070556309362279) ('OTHER', 3, 0.0020352781546811396)
Yennefer : ('PERSON', 1139, 0.993025283347864) ('LOCATION', 7, 0.006102877070619006) ('OTHER', 1, 0.0008718395815170009)
Milva : ('PERSON', 477, 0.9173076923076923) ('LOCATION', 34, 0.06538461538461539) ('OTHER', 5, 0.009615384615384616) ('CONSUMER_GOOD', 2, 0.0038461538461538464) ('ORGANIZATION', 2, 0.0038461538461538464)
Nilfgaard : ('PERSON', 240, 0.6233766233766234) ('LOCATION', 134, 0.34805194805194806) ('ORGANIZATION', 11, 0.02857142857142857)
Triss : ('PERSON', 368, 0.997289972899729)

# Weaviate configuration

In [33]:
import weaviate
weaviate_config = {
    'http_host': "192.168.1.103",
    'http_port': 8080,
    'http_secure': False,
    'grpc_host': "192.168.1.103",
    'grpc_port': 50051,
    'grpc_secure': False
}

In [34]:
with weaviate.connect_to_custom(**weaviate_config) as client:
    print(client.get_meta())

{'hostname': 'http://[::]:8080', 'modules': {}, 'version': '1.24.0-rc.0'}


In [35]:
with weaviate.connect_to_custom(**weaviate_config) as client:
    print(client.collections.delete_all())

None


In [36]:
import weaviate.classes.config as wc

with weaviate.connect_to_custom(**weaviate_config) as client:
    if not 'Book_metadata' in client.collections.list_all().keys():
        client.collections.create(
            name='Book_metadata',
            properties=[
                wc.Property(name='identifier', data_type=wc.DataType.TEXT),
                wc.Property(name='title', data_type=wc.DataType.TEXT),
                wc.Property(name='language', data_type=wc.DataType.TEXT),
                wc.Property(name='creator', data_type=wc.DataType.TEXT),
            ]
        )
    if not 'Book_parts' in client.collections.list_all().keys():
        client.collections.create(
            name='Book_parts',
            properties=[
                wc.Property(name='book_id', data_type=wc.DataType.TEXT),
                wc.Property(name='parent_ids', data_type=wc.DataType.TEXT_ARRAY),
                wc.Property(name='identifiers', data_type=wc.DataType.TEXT_ARRAY),
                wc.Property(name='play_orders', data_type=wc.DataType.NUMBER_ARRAY),
                wc.Property(name='labels', data_type=wc.DataType.TEXT_ARRAY),
                wc.Property(name='content_path', data_type=wc.DataType.TEXT),
                wc.Property(name='content_ids', data_type=wc.DataType.TEXT_ARRAY)
            ]
        )
    if not 'Chunks_250_50' in client.collections.list_all().keys():
        client.collections.create(
            name='Chunks_250_50',
            properties=[
                wc.Property(name='parent_ids', data_type=wc.DataType.TEXT_ARRAY),
                wc.Property(name='chunk_number', data_type=wc.DataType.NUMBER),
                wc.Property(name='content', data_type=wc.DataType.TEXT),
            ]
        )
    if not 'Chunks_1000_100' in client.collections.list_all().keys():
        client.collections.create(
            name='Chunks_1000_100',
            properties=[
                wc.Property(name='parent_ids', data_type=wc.DataType.TEXT_ARRAY),
                wc.Property(name='chunk_number', data_type=wc.DataType.NUMBER),
                wc.Property(name='content', data_type=wc.DataType.TEXT),
            ]
        )

# Embedding

In [37]:
from dotenv import load_dotenv
from pprint import pp
from openai import OpenAI
import os
load_dotenv()

def embedd_text_chunks(text_chunks):
    client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
    response = client.embeddings.create(
        input=text_chunks,
        model="text-embedding-3-small",
        dimensions=1536
    )
    
    return [list(elem.embedding) for elem in response.data]
    


# Chunking

In [38]:
import re

with open("../../data/extracted_books/Sorceleur - L'Integrale - Andrzej Sapkowski.json") as f:
    data = json.load(f)

text = re.sub(r'(\n{3,})', r'\n\n\n', data['data'][2]['children'][1]['content'])


In [39]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n\n", "\n\n", "\n", ".", ",", " ", ""],
    keep_separator=False,
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function=len
)

chunks = splitter.split_text(text)
for chunk in chunks:
    print(chunk)
    print('---')

L
E
S
ORCELEUR

I

On raconta par la suite que l’homme était arrivé par le nord, par la porte des Cordiers. Il allait à pied, menant par la bride son cheval chargé de bagages. L’après-midi était bien avancé, cordiers et bourreliers avaient déjà fermé leurs échoppes, la ruelle était déserte. En dépit de la chaleur, l’homme portait un manteau noir jeté sur ses épaules. Il attirait l’attention.

Il s’arrêta devant l’auberge Au vieux Narakort . Il resta planté là quelques minutes, à écouter le brouhaha des conversations. L’auberge, comme d’habitude à cette heure, était noire de monde.

L’inconnu n’entra pas au Vieux Narakort . Il entraîna son cheval plus loin, vers le bas de la rue, où se trouvait un autre cabaret, plus petit, qui s’appelait Au Renard . Le cabaret était vide. Il n’avait pas très bonne réputation.
---
Le patron leva la tête de son tonneau de cornichons marinés pour toiser son client. L’étranger, qui n’avait pas ôté son manteau, se tenait devant le comptoir ; raide, figé, il

# Weaviate insertion

In [40]:
from weaviate.util import generate_uuid5

metadata = data['metadata']
content = data['data']

book_uuid = generate_uuid5(metadata['identifier'])

with weaviate.connect_to_custom(**weaviate_config) as client:
    book_metadata_collection = client.collections.get('Book_metadata')
    book_parts_collection = client.collections.get('Book_parts')
    chunks_1000_100_collection = client.collections.get('Chunks_1000_100')

    with book_metadata_collection.batch.dynamic() as batch:
        book_metadata_obj = {
            "identifier": book_uuid,
            "title": metadata['title'],
            "language": metadata['language'],
            "creator": metadata['creator']
        }
        batch.add_object(
            properties=book_metadata_obj,
            uuid=book_uuid,
        )
    
    if len(book_metadata_collection.batch.failed_objects) > 0:
        print(f"Failed to import : {book_metadata_collection.batch.failed_objects}")
    
    # Iterating over book parts
    def insert_book_part_recursive(book_part: dict, parent_ids: list[str]):
        print(book_part["label"])
        with book_parts_collection.batch.dynamic() as batch:
            obj = {
                "book_id": book_uuid,
                "parent_ids": parent_ids,
                "identifiers": book_part['id'],
                "play_orders": [int(elem) for elem in book_part['playorder']],
                "labels": book_part['label'],
                "content_path": book_part['content_path'],
                "content_ids": [elem if elem is not None else "" for elem in book_part['content_id']]
            }
            batch.add_object(
                properties=obj,
                uuid=generate_uuid5(obj),
            )
    
        if len(book_parts_collection.batch.failed_objects) > 0:
            print(f"Failed to import : {book_parts_collection.batch.failed_objects}")
        
        content = re.sub(r'(\n{3,})', r'\n\n\n', book_part['content'].strip())
        chunks = splitter.split_text(content)
        if not chunks:
            chunks = [" "]
        embeddings = embedd_text_chunks(chunks)

        with chunks_1000_100_collection.batch.dynamic() as batch:
            for i, chunk in enumerate(chunks):
                obj = {
                    "parent_ids": book_part['id'],
                    "chunk_number": i,
                    "content": chunk
                }
                batch.add_object(
                    properties=obj,
                    uuid=generate_uuid5(obj),
                    vector=embeddings[i]
                )
    
        if len(chunks_1000_100_collection.batch.failed_objects) > 0:
            print(f"Failed to import : {chunks_1000_100_collection.batch.failed_objects}")
        
        for child in book_part['children']:
            insert_book_part_recursive(child, book_part['id'])

    for child in content:
        insert_book_part_recursive(child, [])

['Couverture']
['Titre']
['1 - Le Dernier Vœu']
['La Voix de la raison 1']
['Le Sorceleur']
['La Voix de la raison 2']
['Un grain de vérité']
['La Voix de la raison 3']
['Le Moindre Mal']
['La Voix de la raison 4']
['Une question de prix']
['La Voix de la raison 5']
['Le Bout du monde']
['La Voix de la raison 6']
['Le Dernier Vœu']
['La Voix de la raison 7']
['2 - L’Épée de la providence']
['Les Limites du possible']
['Éclat de glace']
['Le Feu éternel']
["Une once d'abnégation"]
["L'Épée de la providence"]
['Quelque chose en plus']
['3 - Le Sang des elfes']
['Chapitre premier']
['Chapitre 2']
['Chapitre 3']
['Chapitre 4']
['Chapitre 5']
['Chapitre 6']
['Chapitre 7']
['4 - Le Temps du mépris']
['Chapitre premier']
['Chapitre 2']
['Chapitre 3']
['Chapitre 4']
['Chapitre 5']
['Chapitre 6']
['Chapitre 7']
['5 - Le Baptême du feu']
['Chapitre premier']
['Chapitre 2']
['Chapitre 3']
['Chapitre 4']
['Chapitre 5']
['Chapitre 6']
['Chapitre 7']
['6 - La Tour de l’Hirondelle']
['Chapitre premie

# Retrieval

In [45]:
import weaviate.classes.query as wq

query = 'Ciri prophétise la mort de Coën'
query_vector = embedd_text_chunks([query])[0]

with weaviate.connect_to_custom(**weaviate_config) as client:
    chunks_1000_100_collection = client.collections.get('Chunks_1000_100')

    response = chunks_1000_100_collection.query.near_vector(
        near_vector=query_vector,
        limit=10,
        return_metadata=wq.MetadataQuery(distance=True),
        #filters=wq.Filter.by_property("content").contains_any(["Coën"])
    )

    for o in reversed(response.objects):
        print(o.properties['content'])
        print('----------', f'{o.metadata.distance:.3f}')

Comme l’annonce cette prédiction : les cadavres seront entassés sur vingt coudées ; sur la terre désertée les loups hurleront, l’homme embrassera la trace de pas d’un autre homme… Malheur à nous !
---------- 0.583
Ah ! se dit Milva, c’est peut-être vrai ce que rabâchent les prêtres, que la fin du monde et le jour du Jugement dernier sont proches. Le monde est en feu, les hommes sont devenus semblables à des loups, ils s’en prennent non seulement aux elfes, mais également aux autres humains, ils menaceraient leur propre frère de leur couteau… Et voilà maintenant un sorceleur qui se mêle de politique et prend part à la rébellion. Un sorceleur ! Dont la vocation est pourtant d’aller de par le monde tuer les monstres nuisibles ! Depuis que le monde est monde, jamais aucun sorceleur ne s’était laissé entraîner en politique, ni dans une guerre. D’ailleurs, il suffit de penser à la légende de ce roi stupide qui voulait transporter de l’eau dans une passoire, prendre un lièvre comme courrier, 